# Azure ML - Sample Batch Prediction Pipeline
- No parallel run step leveraged

This notebook demonstrates creation and execution of an Azure ML pipeline designed to load data from a remote source, to make predictions against that data using a previously registered ML model, and finally save that data  

In [1]:
experiment_folder = 'email-classification-inference-pipeline'
cluster_name = "cpu-cluster"

## Connect to workspace

### Import Required Packages

In [2]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData, PipelineEndpoint
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig

In [4]:
import azureml.core
from azureml.core import Workspace, Dataset
# Connect to AML Workspace
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

# Get the default datastore
default_ds = ws.get_default_datastore()

Ready to use Azure ML 1.38.0 to work with mlopsdev


In [5]:
import os
# Create a folder for the pipeline step files
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder)

email-classification-inference-pipeline


In order to inference, we need a dataset to inference on, so we will load into the inference folder location our data

In [6]:
#Create a tabular dataset from the path on the datastore (this may take a short while)
default_ds.upload_files(files=['./datasets/spaminference.csv'], # Upload the diabetes csv files in /data
                        target_path= 'spam-data', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)
    
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'spam-data/spaminference.csv'))

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(20).to_pandas_dataframe()

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Uploading ./datasets/spaminference.csv
Uploaded ./datasets/spaminference.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,labels,text
0,spam,This message is free. Welcome to the new & imp...
1,ham,"Excellent, I'll see what riley's plans are"
2,ham,I will see in half an hour
3,spam,You've won tkts to the EURO2004 CUP FINAL or å...
4,ham,Ew are you one of them?
5,ham,Also hi wesley how've you been
6,ham,Ah you see. You have to be in the lingo. I wil...
7,spam,"Loan for any purpose å£500 - å£75,000. Homeown..."
8,spam,Update_Now - 12Mths Half Price Orange line ren...
9,ham,Imagine Life WITHOUT ME... see.. How fast u ar...


### Create Compute Resources

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

print('trying to create: ' + cluster_name)

try:
    # Check for existing compute target
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2, idle_seconds_before_scaledown=1800)
        compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
        compute_target.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

trying to create: cpu-cluster
Found existing cluster, use it.


## Run Configuration
Create configuration for the running pipeline.  The RunConfiguration defines the environment used in the python steps

In [8]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

run_config = RunConfiguration()
run_config.docker.use_docker = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib','pandas','pip'],
                                                                            pip_packages=['azureml-sdk','numpy', 'joblib', 'sklearn' ])

## Define Output Datasets

Below are the configuration for datasets that will be passed between steps in our pipelien.  Note, in all cases we specifiy the datastore that should hold the datasets and wheather they should be registered following step completion or not.  This can optionally be disabled by removing the register_on_complete() call


In [9]:
inferencing_dataset = OutputFileDatasetConfig(name= 'email_inferencing_dataset', destination=(default_ds, 'inferencing_dataset/{run-id}')).read_delimited_files().register_on_complete(name= 'email_inferencing_data')
scored_dataset      = OutputFileDatasetConfig(name= 'email_scored_dataset', destination=(default_ds, 'scored_dataset/{run-id}')).read_delimited_files().register_on_complete(name= 'email_scored_data')

# Define Pipeline Parameters

PipelineParameter objects serve as variable inputs to an Azure ML pipeline and can be specified at runtime. Multiple pipeline parameters can be created and used. Included here are multiple sample pipeline parameters (get_data_param_*) to highlight how parameters can be passed into and consumed by various pipeline steps.

In [10]:
inference_data_location_parm = PipelineParameter(name='inference_data_location', default_value= 'spam-data')
model_name_parm              = PipelineParameter(name='model_name', default_value= 'email_classifier')
get_data_param_2             = PipelineParameter(name='get_data_param_2', default_value='value_2')
get_data_param_3             = PipelineParameter(name='get_data_param_3', default_value='value_3')

In [11]:
import os, shutil
folder_name = 'batch-inferencing'
script_folder = os.path.join(os.getcwd(), folder_name)
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/EmailClass/email_parallelRun_classificationbatch_pipeline/batch-inferencing


In [12]:
%%writefile $script_folder/get_inferencing_data.py

  
from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath
import pandas as pd
import os
import argparse


#Parse input arguments
parser = argparse.ArgumentParser("Get Inferencing Data")
parser.add_argument('--inference_data_location', type=str, required=True)
parser.add_argument('--get_data_param_2', type=str, required=True)
parser.add_argument('--get_data_param_3', type=str, required=True)
parser.add_argument('--inferencing_dataset', dest='inferencing_dataset', required=True)

# Note: the get_data_param args below are included only as an example of argument passing.
# They are not consumed in the code sample shown here.
args, _ = parser.parse_known_args()

inference_data_location = args.inference_data_location
get_data_param_2 = args.get_data_param_2
get_data_param_3 = args.get_data_param_3
inferencing_dataset = args.inferencing_dataset

print(str(type(inferencing_dataset)))
print(inferencing_dataset)

#Get current run
current_run = Run.get_context()

#Get associated AML workspace
ws = current_run.experiment.workspace

#Get default datastore
ds = ws.get_default_datastore()

# Get the default datastore
default_ds = ws.get_default_datastore()

#spam-data/spaminference.csv
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, inference_data_location + '/spaminference.csv'))

# Register the tabular dataset
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                name= 'email-tabular-dataset-raw',
                                description='email data',
                                tags = {'format':'csv'},
                                create_new_version=True)
    print('Dataset registered.')
except Exception as ex:
        print(ex)
        
df = tab_data_set.to_pandas_dataframe()

print('dataset shape = ' + str(df.shape))
print('saving inferencing data: ' + inferencing_dataset)

# Save dataset for consumption in next pipeline step
os.makedirs(inferencing_dataset, exist_ok=True)
df.to_csv(os.path.join(inferencing_dataset, 'inferencing_data.csv'), index=False)

Writing /mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/EmailClass/email_parallelRun_classificationbatch_pipeline/batch-inferencing/get_inferencing_data.py


In [13]:
%%writefile $script_folder/score_inferencing_data.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.core.model import Model
from azureml.data.datapath import DataPath
import pandas as pd
import os
import argparse
import joblib
import json
import joblib
import numpy as np
from azureml.core.model import Model
import time
import pandas as pd
import azureml.core
from azureml.core import Workspace, Dataset
import os
import math


# Parse input arguments
parser = argparse.ArgumentParser("Score Inferencing Data")
parser.add_argument('--model_name_parm', type=str, required=True)
parser.add_argument('--scored_dataset', dest='scored_dataset', required=True)

args, _ = parser.parse_known_args()
model_name = args.model_name_parm
scored_dataset = args.scored_dataset

# Get current run
current_run = Run.get_context()

# Get associated AML workspace
ws = current_run.experiment.workspace

# Get default datastore
ds = ws.get_default_datastore()


inferencing_dataset = current_run.input_datasets['email_inferencing_data']
inferencing_data_df = inferencing_dataset.to_pandas_dataframe()


print('inferencing data df shape:' + str(inferencing_data_df.shape))

#drop columns not in model
#X = data['text']
col_list = ['text']
inferencing_data_df = inferencing_data_df[col_list]

print('model_name' + model_name)

# Get model from workspace - the code below will always retrieve the latest version of the model; specific versions can be targeted.
model_list = Model.list(ws, name=model_name, latest=True)
model_path = model_list[0].download(exist_ok=True)
model = joblib.load(model_path)


print(inferencing_data_df.shape)


X = inferencing_data_df['text']
# Make predictions with new dataframe
predictions = model.predict(X)

print('made predictions')

print(predictions)


inferencing_data_df['Predictions']=predictions

print(inferencing_data_df.head(5))


# Save scored dataset
os.makedirs(scored_dataset, exist_ok=True)
print(scored_dataset)

os.makedirs(scored_dataset, exist_ok=True)
print(scored_dataset)
inferencing_data_df.to_csv(os.path.join(scored_dataset, 'scored_data.csv'), index=False)


Writing /mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/EmailClass/email_parallelRun_classificationbatch_pipeline/batch-inferencing/score_inferencing_data.py


In [14]:
%%writefile $script_folder/publish_scored_data.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath
import pandas as pd
import os
import argparse


# Get current run
current_run = Run.get_context()

# Get associated AML workspace
ws = current_run.experiment.workspace

# Get default datastore
ds = ws.get_default_datastore()

# Get inferencing dataset
scored_dataset = current_run.input_datasets['email_scored_data']
scored_data_df = scored_dataset.to_pandas_dataframe()

# Save dataset to ./outputs dir
os.makedirs('./outputs', exist_ok=True)
scored_data_df.to_csv(os.path.join('outputs', 'scored_data.csv'), index=False)


Writing /mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox/code/Users/babal/EmailClass/email_parallelRun_classificationbatch_pipeline/batch-inferencing/publish_scored_data.py


In [15]:
get_inferencing_data_step = PythonScriptStep(
    name='Get Inferencing Data',
    script_name='get_inferencing_data.py',
    arguments=[
        '--inference_data_location', inference_data_location_parm,
        '--get_data_param_2', get_data_param_2,
        '--get_data_param_3', get_data_param_3,
        '--inferencing_dataset', inferencing_dataset
    ],
    outputs=[inferencing_dataset],
    compute_target=compute_target,
    source_directory=folder_name,
    allow_reuse=False,
    runconfig=run_config
)

score_inferencing_data_step = PythonScriptStep(
    name='Score Inferencing Data',
    script_name='score_inferencing_data.py',
    arguments=[
        '--model_name_parm', model_name_parm,
        '--scored_dataset', scored_dataset
    ],
    inputs=[inferencing_dataset.as_input(name= 'email_inferencing_data')],
    outputs=[scored_dataset],
    compute_target=compute_target,
    source_directory=folder_name,
    allow_reuse=False,
    runconfig=run_config
)

publish_scored_data_step = PythonScriptStep(
    name='Publish Scored Data',
    script_name='publish_scored_data.py',
    inputs=[scored_dataset.as_input(name= 'email_scored_data')],
    compute_target=compute_target,
    source_directory=folder_name,
    allow_reuse=False,
    runconfig=run_config
)

# Create Pipeline

Create an Azure ML Pipeline by specifying the steps to be executed. Note: based on the dataset dependencies between steps, exection occurs logically such that no step will execute unless all of the necessary input datasets have been generated.

In [16]:
pipeline = Pipeline(workspace=ws, steps=[get_inferencing_data_step, score_inferencing_data_step, publish_scored_data_step])

In [17]:
experiment = Experiment(ws,  '01-email-inference-pipeline')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)

Created step Get Inferencing Data [1057e6ce][2a13c70a-51fe-4a40-82a1-20f3d50f79d8], (This step will run and generate new outputs)
Created step Score Inferencing Data [4e0019d7][c5b8fd26-54c1-4cbe-b44f-5758727e0690], (This step will run and generate new outputs)
Created step Publish Scored Data [c1389da1][a1d06f21-8203-40d8-9af2-396a654540da], (This step will run and generate new outputs)
Submitted PipelineRun cba94efa-f494-46d4-97f0-a377fccf7b48
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/cba94efa-f494-46d4-97f0-a377fccf7b48?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRunId: cba94efa-f494-46d4-97f0-a377fccf7b48
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/cba94efa-f494-46d4-97f0-a377fccf7b48?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Stat

Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


# Publish Pipeline

Create a published version of pipeline that can be triggered via a REST API call

In [ ]:
# published_pipeline = pipeline.publish(name = 'Email Batch Inferencing Pipeline',
#                                      description = 'Pipeline that generates batch predictions using a registered trained model.',
#                                      continue_on_step_failure = False)

In [ ]:
# published_pipeline

In [ ]:
# rest_endpoint = published_pipeline.endpoint
# print(rest_endpoint)

In [ ]:
# from azureml.core.authentication import InteractiveLoginAuthentication

# interactive_auth = InteractiveLoginAuthentication()
# auth_header = interactive_auth.get_authentication_header()
# print('Authentication header ready.')

In [ ]:
# import requests

# rest_endpoint = published_pipeline.endpoint
# response = requests.post(rest_endpoint, 
#                          headers=auth_header, 
#                          json={"ExperimentName": user + "rest-api-diabetes-batch"})
# run_id = response.json()["Id"]
# run_id

In [ ]:
# from azureml.pipeline.core.run import PipelineRun
# from azureml.widgets import RunDetails

# published_pipeline_run = PipelineRun(ws.experiments[user + "rest-api-diabetes-batch"], run_id)

# # Block until the run completes
# published_pipeline_run.wait_for_completion(show_output=True)